In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow
tensorflow.__version__

'2.6.0'

In [3]:
tensorflow.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

## Load data

In [6]:
num_classes = 100
img_height = 32
img_width = 32
num_channels = 3
input_shape = (img_height, img_width, num_channels)

In [7]:
train_df = pd.read_csv("/content/drive/MyDrive/Lab1 part2/1/train_data.csv", header=None)
train_df = np.array(train_df)
train_df = train_df.reshape(-1, img_height, img_width, num_channels)
train_df.shape

(10000, 32, 32, 3)

In [8]:
train_label = pd.read_csv("/content/drive/MyDrive/Lab1 part2/1/train_label.csv")
train_label.head()

,id,label
0,0,65
1,1,10
2,2,71
3,3,0
4,4,67


In [9]:
train_label.shape

(10000, 2)

In [10]:
x_train, x_val, y_train, y_val = train_test_split(train_df, train_label.label, test_size = 0.08, random_state=20, stratify=train_label.label)
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)

In [13]:
test_df = pd.read_csv("/content/drive/MyDrive/Lab1 part2/1/test_data.csv", header=None)
test_df = np.array(test_df)
test_df = test_df.reshape(-1, img_height, img_width, num_channels)
test_df.shape

(2000, 32, 32, 3)

## Defining model 

In [32]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(100, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_22 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_23 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 16, 16, 64)       

In [34]:
validation_loss, validation_accuracy = model.evaluate(x_val, y_val, verbose = 0)
print("Validation accuracy = %0.2f%% \t Validation Loss = %0.4f" %(100 * validation_accuracy, validation_loss))

Validation accuracy = 1.13% 	 Validation Loss = 1260.4321


In [35]:
epochs = 400

datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True )
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',patience=30, restore_best_weights=True)

In [38]:
steps = int(x_train.shape[0] / 64)
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=64), steps_per_epoch=steps,validation_data=(x_val, y_val), epochs=epochs, verbose=1, callbacks=[es])

Epoch 1/400
143/143 [==============================] - 9s 54ms/step - loss: 5.6274 - accuracy: 0.0234 - val_loss: 6.8458 - val_accuracy: 0.0437
Epoch 2/400
143/143 [==============================] - 8s 55ms/step - loss: 4.7690 - accuracy: 0.0484 - val_loss: 4.0262 - val_accuracy: 0.1050
Epoch 3/400
143/143 [==============================] - 8s 54ms/step - loss: 4.4076 - accuracy: 0.0715 - val_loss: 3.8822 - val_accuracy: 0.1200
Epoch 4/400
143/143 [==============================] - 8s 54ms/step - loss: 4.2198 - accuracy: 0.0871 - val_loss: 3.7630 - val_accuracy: 0.1363
Epoch 5/400
143/143 [==============================] - 8s 53ms/step - loss: 4.0492 - accuracy: 0.1010 - val_loss: 3.7418 - val_accuracy: 0.1425
Epoch 6/400
143/143 [==============================] - 8s 54ms/step - loss: 3.9127 - accuracy: 0.1147 - val_loss: 3.8130 - val_accuracy: 0.1500
Epoch 7/400
143/143 [==============================] - 8s 56ms/step - loss: 3.8033 - accuracy: 0.1271 - val_loss: 3.6755 - val_accuracy:

### Calculate accuracy 

In [39]:
train_loss, train_accuracy = model.evaluate(x_train, y_train, verbose = 0)
validation_loss, validation_accuracy = model.evaluate(x_val, y_val, verbose = 0)

print("Training accuracy = %0.2f%% \t Training Loss = %0.4f" %(100 * train_accuracy, train_loss))
print("Validation accuracy = %0.2f%% \t Validation Loss = %0.4f" %(100 * validation_accuracy, validation_loss))

Training accuracy = 79.17% 	 Training Loss = 0.7354
Validation accuracy = 48.12% 	 Validation Loss = 2.0853


In [40]:
pred = model.predict(test_df)

In [41]:
df= pd.DataFrame(np.argmax(pred, axis=1), columns=["label"])
df.insert(0, "id", np.arange(len(ans)))
df.shape

(2000, 2)

In [42]:
df.to_csv("attempt2.1.2.csv", index=False)

In [43]:
df

,id,label
0,0,77
1,1,65
2,2,17
3,3,43
4,4,20
...,...,...
1995,1995,4
1996,1996,93
1997,1997,88
1998,1998,1
